In [7]:
from Trainer import Trainer
from Evaluator import Evaluator
from Model import Net
import torch
from kaggle_environments import make, evaluate
import numpy as np

episodes = 2000
batch_size = 32
discount = 0.99
hidden_dim = 300
experienceSize = 20000
epsilon_min_after = 1000
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
trainer = Trainer(hidden_dim, experienceSize, discount, batch_size, device)
trainer.load()


TypeError: __init__() missing 1 required positional argument: 'writer'

In [3]:
env = make("connectx", debug=True)
env.render()
env.reset()
# Play as the first agent against default "random" agent.
#env.run([eval.agent, "random"])
#env.render(mode="ipython", width=500, height=450)
def get_win_percentages(agent1, agent2, n_rounds=100):
    # Use default Connect Four setup
    config = {'rows': 6, 'columns': 7, 'inarow': 4}
    # Agent 1 goes first (roughly) half the time          
    outcomes = evaluate("connectx", [agent1, agent2], config, [], n_rounds//2)
    # Agent 2 goes first (roughly) half the time      
    outcomes += [[b,a] for [a,b] in evaluate("connectx", [agent2, agent1], config, [], n_rounds-n_rounds//2)]
    print("Agent 1 Win Percentage:", np.round(outcomes.count([1,-1])/len(outcomes), 2))
    print("Agent 2 Win Percentage:", np.round(outcomes.count([-1,1])/len(outcomes), 2))
    print("Number of Invalid Plays by Agent 1:", outcomes.count([None, 0]))
    print("Number of Invalid Plays by Agent 2:", outcomes.count([0, None]))
get_win_percentages(eval.agent, "random",100)

Agent 1 Win Percentage: 0.77
Agent 2 Win Percentage: 0.23
Number of Invalid Plays by Agent 1: 0
Number of Invalid Plays by Agent 2: 0


In [23]:
from torch.utils.tensorboard import SummaryWriter
env = make("connectx", debug=True)
env.render()
env.reset()
writer = SummaryWriter('test_run')
trainer = Trainer(hidden_dim, experienceSize, discount, batch_size, device, writer)
def agent(observation, configuration):
    with torch.no_grad():
        net = Net(configuration.columns * configuration.rows, hidden_dim,configuration.columns).to(device)
        state = torch.tensor(observation['board'], dtype=torch.float)
        reshaped = trainer.reshape(state)
        action = trainer.takeAction(net(reshaped).view(-1), reshaped, 0, False)
        print(action)
        return action
# Play as the first agent against default "random" agent.
Evaluator(100, trainer).winPercentage(1)

Agent 1 Win Percentage: 0.0
Agent 2 Win Percentage: 1.0
Number of Invalid Plays by Agent 1: 0
Number of Invalid Plays by Agent 2: 0
